<center>

**CRIPTOGRAFÍA DE LLAVE PÚBLICA**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

# **🗝️RSA🔑**

<p align="center">
    <img src="https://www.ssldragon.com/wp-content/uploads/2025/05/RSA-encryption-process-1024x574.webp"width="400">
</p>

<div align="justify">

RSA es el primer esquema de cifrado asimétrico práctico. Se basa en la dificultad de factorizar grandes números enteros. Cada usuario dispone de:

- Una clave pública $(N, e)$ que puede compartir abiertamente.
- Una clave privada $(N, d)$ que debe guardarse en secreto.

Con la clave pública se cifra, y solo con la privada se descifra.

---

**Fundamentación matemática**

**Elección de primos y módulo**

- Seleccionamos dos primos grandes  $p, q$ por ejemplo, de 1024 bits cada uno

- Calculamos el módulo  $N \;=\; p \times q$

- El grupo multiplicativo módulo $N$ es $\mathbb{Z}_N^* = \{\,x: 1 \le x < N,\;\gcd(x,N)=1\}$


**Función totiente de Euler**

Definimos $\varphi(N) = \bigl|\mathbb{Z}_N^*\bigr| = (p-1)\,(q-1)$

Esta función mide cuántos enteros menores que $N$ son coprimos con él.

**Exponentes de cifrado y descifrado**

- Exponente público $e$: Elegimos un entero $e$ tal que $1 < e < \varphi(N),
  \quad\gcd(e,\varphi(N)) = 1.$

- Exponente privado $d$: Es el inverso multiplicativo de $e$ módulo $\varphi(N)$: $d \equiv e^{-1} \pmod{\varphi(N)},\quad e\,d \equiv 1 \pmod{\varphi(N)}$

---

**Algoritmos Básicos**

**Generación de claves**

- Elegir $p,q$ primos grandes.  

- Calcular
   
     $N = p\,q,\quad
     \varphi(N) = (p-1)(q-1)$

- Seleccionar $e$ coprimo con $\varphi(N)$  

- Calcular $d$ tal que  $\;e\,d \equiv 1 \pmod{\varphi(N)}$  

- Exportar

   - Clave pública: $\bigl(N,e\bigr)$  
   - Clave privada: $\bigl(N,d\bigr)$

**Cifrado**

Para cifrar un mensaje $m$ (representado como un entero con $0 \le m < N$:

$$\boxed{c \;=\; m^e \bmod N}$$

**Descifrado**

Para descifrar el ciphertext $c$:

$$\boxed{m \;=\; c^d \bmod N}$$


---

**Estructura Algebraica**

- RSA actúa en el anillo $\mathbb{Z}/N\mathbb{Z}$.  
- Las operaciones clave son potencias modulares, que aprovechan la reducción rápida (algoritmo de exponenciación binaria).  
- La seguridad descansa en que, sin factorizar $N$, calcular $d$ a partir de $(N,e)$ equivale a resolver un problema de factorización, para el cual no existe algoritmo polinómico conocido.

---

**Propiedades Criptográficas**

- **Confidencialidad**  
   Solo quien posee $d$ puede recuperar $m.  

- **Integridad y autenticidad (firma digital)**  
   - Firma: $s = m^d \bmod N$  
   - Verificación: $m = s^e \bmod N$

- **No repudio**  
   
   Una firma RSA no puede ser forjada sin el conocimiento de $d$.  

- **Resistencia cuántica**  
   La factorización large $N$ es vulnerable a Shor, pero con parámetros muy grandes (4096 bits+) se extiende la ventana de seguridad.





</div>

**📬Instalar paquetes📥**

In [16]:

!pip install pycryptodome

**📥Importaciones📦**

In [17]:

import hashlib
import base64
import ipywidgets as wd
from IPython.display import display, clear_output
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

**👨‍💻Implementación👩‍💻**

In [18]:

def make_randfunc(passphrase: str):

    seed = hashlib.sha256(passphrase.encode('utf-8')).digest()
    counter = 0

    def randfunc(n: int) -> bytes:
        nonlocal counter
        output = b''
        while len(output) < n:
            # Concatenamos semilla + contador para variar cada bloque
            data = seed + counter.to_bytes(8, 'big')
            digest = hashlib.sha256(data).digest()
            output += digest
            counter += 1
        return output[:n]

    return randfunc


def rsa_encrypt(message: str, pubkey_pem: bytes) -> str:
    rsa_key = RSA.import_key(pubkey_pem)
    cipher = PKCS1_OAEP.new(rsa_key)
    ciphertext = cipher.encrypt(message.encode('utf-8'))
    return base64.b64encode(ciphertext).decode('ascii')

def rsa_decrypt(cipher_b64: str, privkey_pem: bytes) -> str:
    rsa_key = RSA.import_key(privkey_pem)
    cipher = PKCS1_OAEP.new(rsa_key)
    ciphertext = base64.b64decode(cipher_b64)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext.decode('utf-8')


**👨‍💻Implementación👩‍💻**

In [22]:

txt_seed  = wd.Text(
    description='Frase semilla:',
    placeholder='Escribe aquí tu texto (p. ej. "pepino")…',
    layout=wd.Layout(width='100%')
)
btn_gen   = wd.Button(
    description='Generar par de claves RSA',
    button_style='info',
    disabled=True
)

txt_pub   = wd.Textarea(
    description='Clave pública PEM:',
    layout=wd.Layout(width='100%', height='120px')
)
txt_priv  = wd.Textarea(
    description='Clave privada PEM:',
    layout=wd.Layout(width='100%', height='120px')
)


txt_msg    = wd.Textarea(
    description='Mensaje claro:',
    layout=wd.Layout(width='65%', height='80px')
)
btn_enc    = wd.Button(description='Cifrar con pública', button_style='success')
txt_cipher = wd.Textarea(
    description='Texto cifrado (B64):',
    layout=wd.Layout(width='100%', height='80px')
)
btn_dec    = wd.Button(description='Descifrar con privada', button_style='warning')
txt_plain  = wd.Textarea(
    description='Texto descifrado:',
    layout=wd.Layout(width='100%', height='80px')
)

out_log    = wd.HTML()


**👨‍💻Implementación👩‍💻**

In [23]:

def on_seed_change(change):
    btn_gen.disabled = not bool(change['new'].strip())

txt_seed.observe(on_seed_change, names='value')


def on_generate(_):
    clear_output(wait=True)
    seed = txt_seed.value.strip()
    randfunc = make_randfunc(seed)
    # Generamos claves determinísticamente
    key = RSA.generate(2048, randfunc=randfunc)
    priv_pem = key.export_key()
    pub_pem  = key.publickey().export_key()
    txt_priv.value = priv_pem.decode('utf-8')
    txt_pub.value  = pub_pem.decode('utf-8')
    out_log.value  = "<b>✔ Claves RSA generadas a partir de tu frase.</b>"
    display(ui)

def on_encrypt(_):
    try:
        pub_pem = txt_pub.value.encode('utf-8')
        ct = rsa_encrypt(txt_msg.value, pub_pem)
        txt_cipher.value = ct
        out_log.value = "<b>✔ Mensaje cifrado con éxito.</b>"
    except Exception as e:
        out_log.value = f"<span style='color:red;'>⚠ Error al cifrar: {e}</span>"

def on_decrypt(_):
    try:
        priv_pem = txt_priv.value.encode('utf-8')
        pt = rsa_decrypt(txt_cipher.value, priv_pem)
        txt_plain.value = pt
        out_log.value = "<b>✔ Mensaje descifrado con éxito.</b>"
    except Exception as e:
        out_log.value = f"<span style='color:red;'>⚠ Error al descifrar: {e}</span>"




**👨‍💻Implementación👩‍💻**

In [24]:

btn_gen.on_click(on_generate)
btn_enc.on_click(on_encrypt)
btn_dec.on_click(on_decrypt)


ui = wd.VBox([
    wd.HTML("<h2 style='color:#663399;'>🔐 RSA Determinístico</h2>"),
    wd.HTML("<b>Paso 1:</b> introduce tu frase semilla"),
    txt_seed,
    btn_gen,
    wd.HTML("<hr><b>Paso 2:</b> claves en formato PEM generadas"),
    wd.HBox([txt_pub, txt_priv]),
    wd.HTML("<hr><b>Paso 3:</b> cifrar / descifrar mensajes"),
    wd.HBox([txt_msg, btn_enc]),
    txt_cipher,
    wd.HBox([btn_dec, txt_plain]),
    out_log
])

display(ui)